In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
import os
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from types import SimpleNamespace
from torch.utils.data import Dataset

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
PATH = os.getcwd()
# PATH = '/content/drive/MyDrive/KGW/dacon/k-water'

In [3]:
config = {
    "INPUT_WINDOW"      : 4 * 24 * 7,   # 1 week, 60 * 24 * 7
    "BATCH_SIZE"        : 128,
    "HIDDEN_DIM_LSTM"   : 16,
    "NUM_LAYERS"        : 1,
    "EPOCHS"            : 1000,
    "LEARNING_RATE"     : 1e-3,
    "PATIENCE"          : 10,
    "RESAMPLE"          : '15min',
    "DEVICE"            : "cpu",
    "SAVE_PATH"         : f"{PATH}/weights/0203"
}

CFG = SimpleNamespace(**config)

CFG.DROPOUT = 0.0 if CFG.NUM_LAYERS < 2 else 0.2

os.makedirs(CFG.SAVE_PATH, exist_ok=True)

In [4]:
df_A = pd.read_csv(f"{PATH}/data/train/TRAIN_A.csv")
df_B = pd.read_csv(f"{PATH}/data/train/TRAIN_B.csv")

In [5]:
df_A['timestamp'] = pd.to_datetime(df_A['timestamp'], format="%y/%m/%d %H:%M")
df_A = df_A.set_index('timestamp')
df_B['timestamp'] = pd.to_datetime(df_B['timestamp'], format="%y/%m/%d %H:%M")
df_B = df_B.set_index('timestamp')

In [6]:
df_A_resample = df_A.resample(f'{CFG.RESAMPLE}').mean()
df_B_resample = df_A.resample(f'{CFG.RESAMPLE}').mean()

In [7]:
df_A_resample = df_A_resample.reset_index('timestamp')
df_B_resample = df_B_resample.reset_index('timestamp')

In [8]:
class TimeSeriesDataset(Dataset):
    def __init__(self, df: pd.DataFrame, stride: int = 1, inference: bool = False):
        """
        Args:
            df: 입력 데이터프레임
            stride: 윈도우 스트라이드
            inference: 추론 모드 여부
        """
        self.inference = inference
        self.column_names = df.filter(regex='^P\\d+$').columns.tolist()
        self.file_ids = df['file_id'].values if 'file_id' in df.columns else None

        if inference:
            self.values = df[self.column_names].values.astype(np.float32)
            self._prepare_inference_data()
        else:
            self._prepare_training_data(df, stride)

    def _normalize_columns(self, data: np.ndarray):
        """벡터화된 열 정규화"""
        mins = data.min(axis=0, keepdims=True)
        maxs = data.max(axis=0, keepdims=True)

        # mins와 maxs가 같으면 전체를 0으로 반환
        is_constant = (maxs == mins)
        if np.any(is_constant):
            normalized_data = np.zeros_like(data)
            normalized_data[:, is_constant.squeeze()] = 0
            return normalized_data

        # 정규화 수행
        return (data - mins) / (maxs - mins)

    def _prepare_inference_data(self):
        """추론 데이터 준비 - 단일 시퀀스"""
        self.normalized_values = self._normalize_columns(self.values)

    def _prepare_training_data(self, df: pd.DataFrame, stride: int):
        """학습 데이터 준비 - 윈도우 단위"""
        self.values = df[self.column_names].values.astype(np.float32)

        # 시작 인덱스 계산 (stride 적용)
        potential_starts = np.arange(0, len(df) - CFG.INPUT_WINDOW, stride)

        # 각 윈도우의 마지막 다음 지점(window_size + 1)이 사고가 없는(0) 경우만 필터링
        accident_labels = df['anomaly'].values
        valid_starts = [
            idx for idx in potential_starts
            if idx + CFG.INPUT_WINDOW < len(df) and  # 범위 체크
            accident_labels[idx + CFG.INPUT_WINDOW] == 0  # 윈도우 다음 지점 체크
        ]
        self.start_idx = np.array(valid_starts)

        # 유효한 윈도우들만 추출하여 정규화
        windows = np.array([
            self.values[i:i + CFG.INPUT_WINDOW]
            for i in self.start_idx
        ])

        # (윈도우 수, 윈도우 크기, 특성 수)로 한번에 정규화
        self.input_data = np.stack([
            self._normalize_columns(window) for window in windows
        ])

    def __len__(self) -> int:
        if self.inference:
            return len(self.column_names)
        return len(self.start_idx) * len(self.column_names)

    def __getitem__(self, idx: int):
        if self.inference:
            col_idx = idx
            col_name = self.column_names[col_idx]
            col_data = self.normalized_values[:, col_idx]
            file_id = self.file_ids[idx] if self.file_ids is not None else None
            return {
                "column_name": col_name,
                "input": torch.from_numpy(col_data).unsqueeze(-1),  # (time_steps, 1)
                "file_id": file_id
            }

        window_idx = idx // len(self.column_names)
        col_idx = idx % len(self.column_names)

        return {
            "column_name": self.column_names[col_idx],
            "input": torch.from_numpy(self.input_data[window_idx, :, col_idx]).unsqueeze(-1)
        }

In [9]:
train_dataset_A = TimeSeriesDataset(df_A[:int(len(df_A)*0.8)], stride=60)
train_dataset_B = TimeSeriesDataset(df_B[:int(len(df_A)*0.8)], stride=60)
train_dataset_A_B = torch.utils.data.ConcatDataset([train_dataset_A, train_dataset_B])

train_loader = torch.utils.data.DataLoader(train_dataset_A_B,
                                            batch_size=CFG.BATCH_SIZE,
                                            shuffle=True)

valid_dataset_A = TimeSeriesDataset(df_A[int(len(df_A)*0.8):], stride=60)
valid_dataset_B = TimeSeriesDataset(df_B[int(len(df_A)*0.8):], stride=60)
valid_dataset_A_B = torch.utils.data.ConcatDataset([valid_dataset_A, valid_dataset_B])

valid_loader = torch.utils.data.DataLoader(valid_dataset_A_B,
                                            batch_size=CFG.BATCH_SIZE,
                                            shuffle=False)

In [10]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(Encoder, self).__init__()

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout, batch_first=True)

    def forward(self, x):
        outputs, hidden = self.lstm(x)
        return outputs, hidden

class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout, output_size):
        super(Decoder, self).__init__()

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x, encoder_hidden):
        outputs, hidden = self.lstm(x, encoder_hidden)
        output = self.linear(outputs.squeeze(1))
        return output, hidden

class LSTMEncoderDecoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout, output_size):
        super(LSTMEncoderDecoder, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size

        self.encoder = Encoder(input_size, hidden_size, num_layers, dropout)
        self.decoder = Decoder(input_size, hidden_size, num_layers, dropout, input_size)

    def forward(self, inputs, teacher_forcing_ratio=0.0):
        batch_size, seq_len = inputs.size(0), inputs.size(1)
        _, hidden = self.encoder(inputs)
        last_hidden = hidden[0][-1]

        outputs = torch.zeros(batch_size, seq_len, self.output_size, dtype=torch.float32).to(inputs.device)

        decoder_input = inputs[:, -1, :].unsqueeze(1)

        for t in range(seq_len):
            output, hidden = self.decoder(decoder_input, hidden)
            outputs[:, t, :] = output

            if random.random() < teacher_forcing_ratio:
                decoder_input = inputs[:, t, :].unsqueeze(1)
            else:
                decoder_input = output.unsqueeze(1)

        reconstructed_hidden = hidden[0][-1]

        return last_hidden, reconstructed_hidden

In [11]:
model = LSTMEncoderDecoder(1, CFG.HIDDEN_DIM_LSTM, CFG.NUM_LAYERS, CFG.DROPOUT, 1).to(CFG.DEVICE)
x = model(torch.randn(128, 672, 1).to(CFG.DEVICE), 1.0)
print(x[0].shape, x[1].shape)

torch.Size([128, 16]) torch.Size([128, 16])


In [12]:
def run(model, train_loader, optimizer, scheduler, criterion, n_epochs, device):
    best_model = {
        "loss": float('inf'),
        "state": None,
        "epoch": 0,
        "epochs_no_improve": 0
    }

    train_losses = []
    valid_losses = []

    for epoch in range(n_epochs):
        model.train()
        train_loss = 0.0

        with tqdm(train_loader, desc=f"Epoch {epoch + 1}/{n_epochs}", unit="batch") as t:
            for batch in t:
                inputs = batch["input"].to(device)
                original_hidden, reconstructed_hidden = model(inputs, 0.0) # [ Batch_size, HIDDEN_DIM_LSTM ]

                loss = criterion(reconstructed_hidden, original_hidden)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                train_loss = train_loss + loss.item()
                t.set_postfix(loss=loss.item())

        avg_train_loss = train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        model.eval()
        valid_loss = 0.0

        with torch.no_grad():
            with tqdm(valid_loader, desc=f"Epoch {epoch + 1}/{n_epochs}", unit="batch") as t:
                for batch in t:
                    inputs = batch["input"].to(device)
                    original_hidden, reconstructed_hidden = model(inputs, 0.0) # [ Batch_size, HIDDEN_DIM_LSTM ]

                    loss = criterion(reconstructed_hidden, original_hidden)

                    valid_loss = valid_loss + loss.item()
                    t.set_postfix(loss=loss.item())
                    
        avg_valid_loss = valid_loss / len(valid_loader)
        valid_losses.append(avg_valid_loss)

        if scheduler:
            scheduler.step(avg_valid_loss)
            lr = scheduler.optimizer.param_groups[0]['lr']

        print(f"Epoch {epoch + 1}/{n_epochs}, Average Train Loss: {avg_train_loss:e}, Average Valid Loss: {avg_valid_loss:e}, Learning Rate: {lr:e}", end=" ")
        
        if avg_valid_loss < best_model["loss"]:
            best_model["state"] = model.state_dict()
            best_model["loss"] = avg_valid_loss
            best_model["epoch"] = epoch + 1
            best_model["epoch_no_improve"] = 0
            torch.save(best_model["state"], f'{CFG.SAVE_PATH}/mse{avg_valid_loss:e}.pt')
            print("Best Model Update & Saved !")
        else:
            best_model["epoch_no_improve"] += 1
            print("")

        if best_model["epoch_no_improve"] >= CFG.PATIENCE:
            print(f"Early stopping at epoch {epoch + 1}. Validation loss did not improve for {CFG.PATIENCE} consecutive epochs.")
            break
        
        if epoch % 10 == 0:
            clear_output()
            
    return train_losses, best_model

In [13]:
MODEL = LSTMEncoderDecoder(1, CFG.HIDDEN_DIM_LSTM, CFG.NUM_LAYERS, CFG.DROPOUT, 1).to(CFG.DEVICE)
criterion = nn.MSELoss()
optimizer = optim.AdamW(MODEL.parameters(), lr=CFG.LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, threshold=1e-16, min_lr=1e-16)

In [14]:
train_losses, valid_losses, best_model = run(
    MODEL,
    train_loader=train_loader,
    optimizer=optimizer,
    scheduler=scheduler,
    criterion=criterion,
    n_epochs=CFG.EPOCHS,
    device=CFG.DEVICE
)

Epoch 1/1000:  14%|█▍        | 23/163 [01:52<11:23,  4.88s/batch, loss=0.00361]


KeyboardInterrupt: 

In [31]:
INFER_MODEL = LSTMEncoderDecoder(1, CFG.HIDDEN_DIM_LSTM, CFG.NUM_LAYERS, CFG.DROPOUT, 1).to(CFG.DEVICE)
INFER_MODEL.load_state_dict(torch.load(f'{PATH}/weights/H128_L1_mse0.0000000002804242.pt', weights_only=True, map_location=CFG.DEVICE)) # best_model["state"]

<All keys matched successfully>

In [32]:
def calculate_and_save_threshold(MODEL, train_loader, percentile=98):
    MODEL.eval()
    train_errors = []
    with torch.no_grad():
        for batch in tqdm(train_loader):
            inputs = batch["input"].to(CFG.DEVICE)
            original_hidden, reconstructed_hidden = MODEL(inputs, 0.0)
            mse_errors = torch.mean((original_hidden - reconstructed_hidden) ** 2, dim=1).cpu().numpy()
            train_errors.extend(mse_errors)

    threshold = np.percentile(train_errors, percentile)

    print(f"Threshold calculated and saved: {threshold}")
    return threshold

THRESHOLD = calculate_and_save_threshold(INFER_MODEL, train_loader)

100%|██████████| 98/98 [12:07<00:00,  7.42s/it]

Threshold calculated and saved: 2.182152458729547e-09


In [33]:
def inference_test_files(MODEL, batch, device='cuda'):
    MODEL.eval()
    with torch.no_grad():
        inputs = batch["input"].to(device)
        original_hidden, reconstructed_hidden = MODEL(inputs)
        reconstruction_loss = torch.mean((original_hidden - reconstructed_hidden) ** 2, dim=1).cpu().numpy()
    return reconstruction_loss

def detect_anomaly(MODEL, test_directory):
    test_files = [f for f in os.listdir(test_directory) if f.startswith("TEST") and f.endswith(".csv")]
    test_datasets = []
    all_test_data = []

    for filename in tqdm(test_files, desc='Processing test files'):
        test_file = os.path.join(test_directory, filename)
        df = pd.read_csv(test_file)
        df['file_id'] = filename.replace('.csv', '')
        individual_df = df[['timestamp', 'file_id'] + df.filter(like='P').columns.tolist()]
        individual_dataset = TimeSeriesDataset(individual_df, inference=True)
        test_datasets.append(individual_dataset)

        all_test_data.append(df)

    combined_dataset = torch.utils.data.ConcatDataset(test_datasets)

    test_loader = torch.utils.data.DataLoader(combined_dataset,  batch_size=128,  shuffle=False)

    reconstruction_errors = []
    for batch in tqdm(test_loader):
        reconstruction_loss = inference_test_files(MODEL, batch, CFG.DEVICE)

        for i in range(len(reconstruction_loss)):
            reconstruction_errors.append({
                "ID": batch["file_id"][i],
                "column_name": batch["column_name"][i],
                "reconstruction_error": reconstruction_loss[i]
            })

    errors_df = pd.DataFrame(reconstruction_errors)

    flag_columns = []
    for column in sorted(errors_df['column_name'].unique()):
        flag_column = f'{column}_flag'
        errors_df[flag_column] = (errors_df.loc[errors_df['column_name'] == column, 'reconstruction_error'] > THRESHOLD).astype(int)
        flag_columns.append(flag_column)

    errors_df_pivot = errors_df.pivot_table(index='ID',
                                          columns='column_name',
                                          values=flag_columns,
                                          aggfunc='first')
    errors_df_pivot.columns = [f'{col[1]}' for col in errors_df_pivot.columns]
    errors_df_flat = errors_df_pivot.reset_index()

    errors_df_flat['flag_list'] = errors_df_flat.loc[:, 'P1':'P' + str(len(flag_columns))].apply(lambda x: x.tolist(), axis=1).apply(lambda x: [int(i) for i in x])
    return combined_dataset, errors_df, errors_df_flat[["ID", "flag_list"]]

In [35]:
C_dataset, C_error_df, C_list = detect_anomaly(INFER_MODEL, test_directory=f"{PATH}/data/test/C")

100%|██████████| 183/183 [20:07<00:00,  6.60s/it]


In [36]:
D_dataset, D_error_df, D_list = detect_anomaly(INFER_MODEL, test_directory=f"{PATH}/data/test/D")

100%|██████████| 129/129 [12:26<00:00,  5.79s/it]


In [39]:
C_D_list = pd.concat([C_list, D_list])

In [40]:
sample_submission = pd.read_csv(f"{PATH}/data/sample_submission.csv")

flag_mapping = C_D_list.set_index("ID")["flag_list"]
sample_submission["flag_list"] = sample_submission["ID"].map(flag_mapping).fillna(sample_submission["flag_list"])

sample_submission.to_csv(f"{PATH}/results/LSTM_ED_H128_L1_mse0.0000000002804242.pt.csv", index=False)